In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
import joblib

# Load data:

In [2]:
data = pd.read_csv('winequality-red.csv', sep=';')
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [3]:
data.shape

(1599, 12)

In [4]:
data.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000,1599.000000
mean,8.319637,0.527821,0.270976,2.538806,0.087467,15.874922,46.467792,0.996747,3.311113,0.658149,10.422983,5.636023
std,1.741096,0.179060,0.194801,1.409928,0.047065,10.460157,32.895324,0.001887,0.154386,0.169507,1.065668,0.807569
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000
25%,7.100000,0.390000,0.090000,1.900000,0.070000,7.000000,22.000000,0.995600,3.210000,0.550000,9.500000,5.000000
50%,7.900000,0.520000,0.260000,2.200000,0.079000,14.000000,38.000000,0.996750,3.310000,0.620000,10.200000,6.000000
75%,9.200000,0.640000,0.420000,2.600000,0.090000,21.000000,62.000000,0.997835,3.400000,0.730000,11.100000,6.000000
max,15.900000,1.580000,1.000000,15.500000,0.611000,72.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000


**Note**: this tutorial doesn't discuss exploratory data analysis in depth but it's a vital part of real-world machine learning. 

# Split data into training and test sets:

In [5]:
# seperate target from training features:
y = data.quality
X = data.loc[:, :'alcohol'] # or: X = data.drop('quality', axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123, stratify=y)

**Note**: It's good practice to **stratify your sample** by the target variable. This will ensure your training set looks similar to your test set, making your evaluation metrics more reliable.

**Definition:**: Stratification is the process of dividing members of the population into homogeneous subgroups before sampling. The strata should define a partition of the population. Then simple random sampling or systematic sampling is applied within each stratum. 

# Declare data preprocessing steps:

**Definition:** 

Standardization is the process of subtracting the mean from each feature value and then dividing by the standard deviation.

Standardization is a common requirement for machine learning tasks. Many algorithms assume that all features are centered around zero and have approximately the same variance. This is to ensure that all features follow the same scale. 

In [7]:
# Notice: we won't actually use the following code!

X_train_scaled = preprocessing.scale(X_train)

print(np.round(X_train_scaled.mean(axis=0)))
print(X_train_scaled.std(axis=0))

[ 0. -0. -0. -0.  0. -0. -0. -0. -0. -0. -0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


We can confirm that the scaled dataset is centered at approximately zero, with unit variance (1).

---

**Standard Normal Distribution:** The standard normal distribution is a special case of the normal distribution. It is the distribution that occurs when a normal random variable has a **mean of zero** and a **standard deviation of one**.

The normal random variable of a standard normal distribution is called a standard score or a $z$ score. Every normal random variable $X$ can be transformed into a $z$ score via the following equation: $$z = (X - \mu) / \sigma$$

where $X$ is a normal random variable, $\mu$ is the mean, and $\sigma$ is the standard deviation.

<img src="https://sphweb.bumc.bu.edu/otlt/MPH-Modules/PH717-QuantCore/PH717-Module6-RandomError/Normal%20Distribution%20deviations.png" width="500" height="500">

---

**Note**:

The reason why we won't use the code above (cell 7) is that we won't be able to perform the exact same transformation on the test set.

We can still scale the test set separately, but we won't be using the same means and standard deviations as we used to transform the training set. That means it wouldn't be a fair representation of how the model pipeline, including the preprocessing steps, would perform on brand new data.

Instead of directly invoking the ```scale``` function, we'll be using a feature in Scikit-Learn called the Transformer API. The Transformer API allows you to "fit" a preprocessing step using the training data the same way you'd fit a model and then use the same transformation on future data sets!

Here's what that process looks like:
1. Fit the transformer on the training set (saving the means and standard deviations)
2. Apply the transformer to the training set (scaling the training data)
3. Apply the transformer to the test set (using the same means and standard deviations)

This makes your final estimate of model performance more realistic, and it allows to insert your preprocessing steps into a cross-validation pipeline (more details below).

In [8]:
# fit the Transformer API:
scaler = preprocessing.StandardScaler().fit(X_train)

Now the scaler object has the saved means and standard deviations for each feature in the training set.

In [9]:
X_train_scaled = scaler.transform(X_train)
print(np.round(X_train_scaled.mean(axis=0)))
print(X_train_scaled.std(axis=0))

[ 0. -0. -0. -0.  0. -0. -0. -0. -0. -0. -0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


We can transform the test set using the exact same means and standard deviations used to transform the training set:

In [10]:
X_test_scaled = scaler.transform(X_test)
 
print(X_test_scaled.mean(axis=0))
print(X_test_scaled.std(axis=0))

[ 0.02776704  0.02592492 -0.03078587 -0.03137977 -0.00471876 -0.04413827
 -0.02414174 -0.00293273 -0.00467444 -0.10894663  0.01043391]
[1.02160495 1.00135689 0.97456598 0.91099054 0.86716698 0.94193125
 1.03673213 1.03145119 0.95734849 0.83829505 1.0286218 ]


**Note:** The scaled features in the test set are not perfectly centered at zero with unit variance! This is exactly what we'd expect, as we're transforming the test set using the means from the training set, not from the test set itself.

In practice, when we set up the cross-validation pipeline, we won't even need to manually fit the Transformer API. Instead, we'll simply declare the class object, like so:

In [11]:
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestRegressor(n_estimators=100))

This is exactly what it looks like: a modeling pipeline that first transforms the data using ```StandardScaler()``` and then fits a model using a random forest regressor.

# Declare hyperparameters to tune:

**Definition:** There are two types of parameters we need to worry about: model parameters and hyperparameters. Models parameters can be learned directly from the data (e.g. regression coefficients), while hyperparameters cannot.

Hyperparameters express "higher-level" structural information about the model, and they are typically set before training the model (e.g. polynomial degree).

**Random Forest hyperparameters:**
Within each decision tree, the computer can empirically decide where to create branches based on either mean-squared-error (MSE) or mean-absolute-error (MAE). The actual branch locations are model parameters.

However, the algorithm does not know which of the two criteria, MSE or MAE, that it should use. The algorithm also cannot decide how many trees to include in the forest. These are examples of hyperparameters that the user must set.

In [12]:
# list tunable hyperparameters:
pipeline.get_params()

{'memory': None,
 'steps': [('standardscaler',
   StandardScaler(copy=True, with_mean=True, with_std=True)),
  ('randomforestregressor',
   RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                         max_depth=None, max_features='auto', max_leaf_nodes=None,
                         max_samples=None, min_impurity_decrease=0.0,
                         min_impurity_split=None, min_samples_leaf=1,
                         min_samples_split=2, min_weight_fraction_leaf=0.0,
                         n_estimators=100, n_jobs=None, oob_score=False,
                         random_state=None, verbose=0, warm_start=False))],
 'verbose': False,
 'standardscaler': StandardScaler(copy=True, with_mean=True, with_std=True),
 'randomforestregressor': RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
 

You can also find a list of all the parameters on the [RandomForestRegressor documentation page](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html). Just note that when it's tuned through a pipeline, you'll need to prepend ```randomforestregressor__``` before the parameter name, as shown above.

In [13]:
# declare hyperparameters to tune:
hyperparameters = {'randomforestregressor__max_features': ['auto', 'sqrt', 'log2'], 
                   'randomforestregressor__max_depth': [None, 5, 3, 1]}

The format should be a Python dictionary where keys are the hyperparameter names and values are lists of settings to try. The options for parameter values can be found on the documentation page.

# Tune model using a cross-validation pipeline:

**Cross-validation (CV) steps:**
1. Split your data into $k$ equal parts, or "folds" (typically $k=10$).
2. Train your model on $k-1$ folds (e.g. the first 9 folds).
3. Evaluate it on the remaining "hold-out" fold (e.g. the 10th fold).
4. Perform steps (2) and (3) k times, each time holding out a different fold.
5. Aggregate the performance across all $k$ folds. This is your performance metric.

<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/K-fold_cross_validation_EN.svg/1280px-K-fold_cross_validation_EN.svg.png' width="500">

**Why is cross-validation important in machine learning?**

Using **only your training set**, you can use CV to evaluate different hyperparameters and estimate their effectiveness. This allows you to keep your test set "untainted" and save it for **a true hold-out evaluation** when you're finally ready to select a model.

For example, you can use CV to tune a random forest model, a linear regression model, and a k-nearest neighbors model, using only the training set. Then, you still have the untainted test set to make your final selection between the model families!

**What is a cross-validation pipeline?**

The best practice when performing CV is to include your data preprocessing steps inside the cross-validation loop. This prevents accidentally tainting your training folds with influential data from your test fold.

**Here's how the CV pipeline looks after including preprocessing steps:**
1. Split your data into k equal parts, or "folds" (typically k=10).
2. Preprocess k-1 training folds.
3. Train your model on the same k-1 folds.
4. Preprocess the hold-out fold using the same transformations from step (2).
5. Evaluate your model on the same hold-out fold.
6. Perform steps (2) - (5) k times, each time holding out a different fold.
7. Aggregate the performance across all k folds. This is your performance metric.

In [14]:
# set up the cross-validation pipeline:
clf = GridSearchCV(pipeline, hyperparameters, cv=10)

# fit and tune model:
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('randomforestregressor',
                                        RandomForestRegressor(bootstrap=True,
                                                              ccp_alpha=0.0,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              max_samples=None,
                            

**Note:** Here, we only apply CV to ```X_train``` and ```y_train```. The purpose of CV is to evaluate and select the best hyperparameter values, not evaluate the performance of the entire model. Instead, we will use the test set (```X_test``` and ```y_test```) to do this later.

**Note:**```GridSearchCV``` essentially performs cross-validation across the entire "grid" (all possible permutations) of hyperparameters. Now, you can see the best set of parameters found using CV:

In [17]:
clf.best_params_

{'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 'log2'}

***Tip:*** *It turns out that in practice, random forests don't actually require a lot of tuning. They tend to work pretty well out-of-the-box with a reasonable number of trees. Even so, these same steps can be used when building any type of supervised learning model.*

# Refit on the entire training set:

After you've tuned your hyperparameters appropriately using cross-validation, you can generally get a small performance improvement by refitting the model on the entire training set.

Conveniently, ```GridSearchCV``` from sklearn will automatically refit the model with the best set of hyperparameters using the entire training set.

This functionality is ON by default, but you can confirm it:

In [18]:
clf.refit

True

# Evaluate model pipeline on test data:

In [21]:
# predict a new set of data:
y_pred = clf.predict(X_test)

# evaluate the model with the imported metrics:
print(r2_score(y_test, y_pred))
print(mean_squared_error(y_test, y_pred))

0.47773012894243005
0.337006875


There are various ways to improve a model. We'll have more guides that go into detail about how to improve model performance, but here are a few quick things to try:
1. Try other regression model families (e.g. regularized regression, boosted trees, etc.).
2. Collect more data if it's cheap to do so.
3. Engineer smarter features after spending more time on exploratory analysis.

**Note**: When you try other families of models, we recommend using the same training and test set as you used to fit the random forest model. That's the best way to get a true apples-to-apples comparison between your models.

# Save model for future use:

In [23]:
# save model to a .pkl file:
joblib.dump(clf, 'rf_regressor.pkl')

# load model from .pkl file:
clf2 = joblib.load('rf_regressor.pkl')
clf2.predict(X_test)

array([6.28, 5.69, 4.97, 5.55, 6.38, 5.51, 5.06, 4.82, 5.01, 5.91, 5.32,
       5.74, 5.66, 5.19, 5.78, 5.76, 6.58, 5.63, 5.72, 6.97, 5.46, 5.58,
       4.99, 6.05, 5.91, 5.02, 5.43, 5.18, 6.15, 5.97, 5.93, 6.43, 5.99,
       5.02, 4.99, 5.91, 5.05, 6.04, 5.05, 6.07, 4.93, 5.96, 6.59, 5.04,
       6.21, 5.26, 5.63, 5.56, 5.11, 6.42, 6.06, 5.41, 5.84, 5.13, 5.61,
       5.54, 5.34, 5.35, 4.98, 5.33, 5.34, 5.17, 5.02, 5.87, 5.93, 5.24,
       6.37, 5.02, 5.06, 6.68, 5.67, 5.6 , 5.18, 5.02, 5.33, 5.98, 5.31,
       5.08, 5.21, 5.29, 6.4 , 5.65, 6.07, 6.4 , 5.07, 6.16, 6.58, 6.28,
       5.54, 5.65, 6.03, 5.52, 6.53, 5.7 , 5.76, 5.75, 6.75, 6.72, 5.56,
       6.77, 5.07, 5.49, 5.12, 6.43, 5.07, 4.81, 5.72, 5.04, 5.72, 5.99,
       5.8 , 5.48, 6.04, 5.38, 5.23, 5.2 , 5.92, 5.05, 5.06, 5.94, 5.85,
       5.11, 5.79, 6.07, 5.29, 5.27, 5.3 , 5.93, 5.48, 5.34, 5.65, 6.21,
       5.09, 5.32, 5.12, 6.38, 5.04, 5.15, 6.67, 5.52, 5.19, 5.01, 5.6 ,
       6.03, 5.36, 5.33, 5.11, 6.43, 5.8 , 5.07, 5.